In [ ]:
#!pip install annoy

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

# Read the CSV file into a DataFrame
games = pd.read_csv("games.csv")

In [2]:
games.drop('Estimated owners', inplace=True, axis=1)
games.drop('Peak CCU', inplace=True, axis=1)
games.drop('Required age', inplace=True, axis=1)
games.drop('DLC count', inplace=True, axis=1)
games.drop('Supported languages', inplace=True, axis=1)
games.drop('Full audio languages', inplace=True, axis=1)
games.drop('Support email', inplace=True, axis=1)
games.drop('Support url', inplace=True, axis=1)
games.drop('Website', inplace=True, axis=1)
games.drop('Windows', inplace=True, axis=1)
games.drop('Mac', inplace=True, axis=1)
games.drop('Linux', inplace=True, axis=1)
games.drop('Screenshots', inplace=True, axis=1)
games.drop('Movies', inplace=True, axis=1)
games.drop('Achievements', inplace=True, axis=1)
games.drop('Notes', inplace=True, axis=1)
games.drop('Score rank', inplace=True, axis=1)
games.drop('Recommendations', inplace=True, axis=1)
games.drop('User score', inplace=True, axis=1)
games.drop('Metacritic score', inplace=True, axis=1)
games.drop('Metacritic url', inplace=True, axis=1)

In [3]:
games["Review Ratio"]=(games["Positive"]/(games["Positive"]+games["Negative"]))
games["Review Sum"]=(games["Positive"]+games["Negative"])
steam_total_reviews=sum(games['Review Sum'])
steam_total_reviews
games['Combined Score'] = games['Review Ratio'] * (games['Review Sum']/steam_total_reviews)

In [4]:
games

,AppID,Name,Release date,Price,About the game,Reviews,Header image,Positive,Negative,Average playtime forever,...,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Review Ratio,Review Sum,Combined Score
0,20200,Galactic Bowling,"Oct 21, 2008",19.99,Galactic Bowling is an exaggerated and stylize...,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,6,11,0,...,0,0,Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports","Indie,Casual,Sports,Bowling",0.352941,17,6.451184e-08
1,655370,Train Bandit,"Oct 12, 2017",0.99,THE LAW!! Looks to be a showdown atop a train....,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,53,5,0,...,0,0,Rusty Moyher,Wild Rooster,"Single-player,Steam Achievements,Full controll...","Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",0.913793,58,5.698546e-07
2,1732930,Jolt Project,"Nov 17, 2021",4.99,Jolt Project: The army now has a new robotics ...,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,0,0,0,...,0,0,Campião Games,Campião Games,Single-player,"Action,Adventure,Indie,Strategy",NaN,NaN,0,NaN
3,1355720,Henosis™,"Jul 23, 2020",5.99,HENOSIS™ is a mysterious 2D Platform Puzzler w...,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,3,0,0,...,0,0,Odd Critter Games,Odd Critter Games,"Single-player,Full controller support","Adventure,Casual,Indie","2D Platformer,Atmospheric,Surreal,Mystery,Puzz...",1.000000,3,3.225592e-08
4,1139950,Two Weeks in Painland,"Feb 3, 2020",0.00,ABOUT THE GAME Play as a hacker who has arrang...,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,50,8,0,...,0,0,Unusual Games,Unusual Games,"Single-player,Steam Achievements","Adventure,Indie","Indie,Adventure,Nudity,Violent,Sexual Content,...",0.862069,58,5.375986e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71711,2159120,SUR5,"Apr 1, 2023",0.00,"Innovative Gameplay SUR5 brings a brand new, f...",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,0,0,0,...,0,0,Katharsis,Katharsis,"Single-player,Multi-player,PvP,Online PvP,Stea...","Free to Play,Indie,Massively Multiplayer,Strat...",NaN,NaN,0,NaN
71712,2125160,Prison Life 2,"Mar 30, 2023",10.79,Core Features Real players only! All Guards an...,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,9,1,0,...,0,0,AO Games,AO Games,"Single-player,Multi-player,PvP,Online PvP,Co-o...","Casual,Indie,RPG,Simulation,Strategy","First-Person,Detective,Social Deduction,Multip...",0.900000,10,9.676775e-08
71713,2318860,Architecture Zeitgeist,"Mar 27, 2023",5.99,Architecture Zeitgeist is a game created with ...,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,1,0,0,...,0,0,Matheus Rudo,Rudo Studios,Single-player,"Casual,Simulation,Early Access","Casual,Simulation,Puzzle,Exploration,Immersive...",1.000000,1,1.075197e-08
71714,2180330,Girl's Way to Survive 2: Bloody Museum,"Apr 2, 2023",2.39,"Development progress: prologue map (finished),...",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,0,0,0,...,0,0,绅士游戏工坊,绅士游戏工坊,Single-player,"Action,Casual,Indie,RPG,Early Access",NaN,NaN,0,NaN


In [6]:
# Preprocess the data
games['Publishers'] = games['Publishers'].fillna('')
games['Genres'] = games['Genres'].fillna('')
games['Features'] = games['Publishers'] + ' ' + games['Genres']

# Create a TF-IDF vectorizer for both combined features and genres
tfidf_vectorizer_combined = TfidfVectorizer()
tfidf_matrix_combined = tfidf_vectorizer_combined.fit_transform(games['Features'])
tfidf_vectorizer_genres = TfidfVectorizer()
tfidf_matrix_genres = tfidf_vectorizer_genres.fit_transform(games['Genres'])

# Build nearest neighbors models for both combined features and genres
knn_combined = NearestNeighbors(n_neighbors=6, metric='cosine')
knn_genres = NearestNeighbors(n_neighbors=6, metric='cosine')
knn_combined.fit(tfidf_matrix_combined)
knn_genres.fit(tfidf_matrix_genres)

# Function to recommend similar games with "Header image" and "Genres"
def recommend_similar_games(game_name, games_df=games, knn_combined=knn_combined, knn_genres=knn_genres, tfidf_matrix_combined=tfidf_matrix_combined, tfidf_matrix_genres=tfidf_matrix_genres):
    # Find the index of the given game in the DataFrame
    game_index = games_df[games_df['Name'] == game_name].index[0]
    
    # Use the nearest neighbors model to find the most similar games for both combined and genres models
    distances_combined, indices_combined = knn_combined.kneighbors(tfidf_matrix_combined[game_index])
    distances_genres, indices_genres = knn_genres.kneighbors(tfidf_matrix_genres[game_index])
    
    # Exclude the input game itself from recommendations for both models
    similar_game_indices_combined = indices_combined[0][1:]
    similar_game_indices_genres = indices_genres[0][1:]
    
    # Get the 5 most similar games for both models
    similar_games_combined = [games_df.loc[i] for i in similar_game_indices_combined]
    similar_games_genres = [games_df.loc[i] for i in similar_game_indices_genres]
    
    # Check if the priority model (combined) has enough recommendations
    if len(similar_games_combined) >= 5:
        recommended_games = similar_games_combined[:5]
    else:
        # Combine recommendations from the priority model and genres model
        combined_recommendations = similar_games_combined + similar_games_genres
        
        # Remove duplicates and return up to 5 recommendations
        recommended_games = list(set(combined_recommendations))[:5]
    
    return pd.DataFrame(recommended_games)

# Example usage
input_game_name = "Galactic Bowling"
recommended_games_df = recommend_similar_games(input_game_name)

if not recommended_games_df.empty:
    print(f"For the game '{input_game_name}', 5 similar recommendations are:")
    for i, (_, game) in enumerate(recommended_games_df.iterrows()):
        print(f"{i + 1}. Name: {game['Name']}, Header image: {game['Header image']}, Genres: {game['Genres']}")
else:
    print(f"No recommendations found for the game '{input_game_name}'.")


For the game 'Galactic Bowling', 5 similar recommendations are:
1. Name: Football Club Simulator - FCS #21, Header image: https://cdn.akamai.steamstatic.com/steam/apps/324090/header.jpg?t=1622140190, Genres: Casual,Simulation,Sports,Strategy
2. Name: Imperivm RTC - HD Edition Great Battles of Rome, Header image: https://cdn.akamai.steamstatic.com/steam/apps/752580/header.jpg?t=1629132365, Genres: Strategy
3. Name: Arena, Header image: https://cdn.akamai.steamstatic.com/steam/apps/717300/header.jpg?t=1558079788, Genres: Free to Play,Indie,Strategy
4. Name: Time Keeper, Header image: https://cdn.akamai.steamstatic.com/steam/apps/1727500/header.jpg?t=1661385291, Genres: Adventure
5. Name: Dragon Lords: 3D Strategy, Header image: https://cdn.akamai.steamstatic.com/steam/apps/661960/header.jpg?t=1505189671, Genres: Free to Play,Massively Multiplayer,Strategy


Calculate the "Review Ratio" for each game.

Consider both genres and the "Review Ratio" for recommendations, prioritizing genres.

Rank the games based on the "Review Ratio" as a secondary factor.

Ensure that the total number of reviews (positive + negative) is also considered to prevent overly biased recommendations based solely on "Review Ratio."

In [ ]:
# Create a reverse index for the "Review Sum" column with the "Name" column
review_sum_reverse_index = dict(zip(games['Name'], games['Review Sum']))

# Now you can access the review sum for a game by its name
game_name = "Super Tennis Blast"
if game_name in review_sum_reverse_index:
    review_sum = review_sum_reverse_index[game_name]
    print(f"Review Sum for '{game_name}': {review_sum}")
else:
    print(f"No review sum found for '{game_name}'")


In [ ]:
# Create a reverse index for the "Review Sum" column with the "Name" column
review_sum_reverse_index = dict(zip(games['Name'], games['Genres']))

# Now you can access the review sum for a game by its name
game_name = "Football Club Simulator - FCS #21"
if game_name in review_sum_reverse_index:
    review_sum = review_sum_reverse_index[game_name]
    print(f"Review Sum for '{game_name}': {review_sum}")
else:
    print(f"Genres for '{game_name}'")

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import pandas as pd

# Preprocess the data
games['Genres'] = games['Genres'].fillna('')
games['Combined Score'] = games['Combined Score'].fillna(0)  # Assuming you have a 'Combined Score' column
games['Features'] = games['Genres'] + ' ' + games['Combined Score'].astype(str)

# Create a TF-IDF vectorizer for the combined features
tfidf_vectorizer_combined = TfidfVectorizer()
tfidf_matrix_combined = tfidf_vectorizer_combined.fit_transform(games['Features'])

# Build a nearest neighbors model for combined features
knn_combined = NearestNeighbors(n_neighbors=30, metric='cosine')  # Increase n_neighbors to allow for more recommendations
knn_combined.fit(tfidf_matrix_combined)

# Function to recommend similar games with "Genres" and "Combined Score" for multiple input games
def recommend_similar_games(input_games, games_df=games, knn_combined=knn_combined, tfidf_matrix_combined=tfidf_matrix_combined):
    recommended_games = []

    for game_name in input_games:
        # Find the index of the given game in the DataFrame
        game_index = games_df[games_df['Name'] == game_name].index[0]

        # Use the nearest neighbors model to find the most similar games based on combined features
        distances_combined, indices_combined = knn_combined.kneighbors(tfidf_matrix_combined[game_index])

        # Get the 5 most similar games based on combined features
        similar_game_indices_combined = indices_combined[0]
        similar_games_combined = [games_df.loc[i] for i in similar_game_indices_combined]

        # Sort the recommendations by the "Combined Score" in descending order
        similar_games_combined = sorted(similar_games_combined, key=lambda x: x['Combined Score'], reverse=True)

        # Filter out the input games themselves and any duplicates
        unique_recommendations = [game for game in similar_games_combined if game['Name'] not in input_games and game['Name'] not in [g['Name'] for g in recommended_games]]

        # Append up to 5 unique recommendations
        recommended_games.extend(unique_recommendations[:5])

    return pd.DataFrame(recommended_games[:5])

# Example usage with two input games
input_games = ["Galactic Bowling", "Circle of Sumo","Dr. Tacocat"]
recommended_games_df = recommend_similar_games(input_games)

if not recommended_games_df.empty:
    print("Recommended games:")
    for i, (_, game) in enumerate(recommended_games_df.iterrows()):
        print(f"Input Game: {input_games[i % len(input_games)]}, Recommendation {i + 1}: Name: {game['Name']}, Header image: {game['Header image']}, Genres: {game['Genres']}, Combined Score: {game['Combined Score']}")
else:
    print("No recommendations found for the input games.")


Recommended games:
Input Game: Galactic Bowling, Recommendation 1: Name: Don't Drop the Bass, Header image: https://cdn.akamai.steamstatic.com/steam/apps/471220/header.jpg?t=1582470274, Genres: Casual,Indie,Sports, Combined Score: 6.451183656726153e-08
Input Game: Circle of Sumo, Recommendation 2: Name: Cosmic Kites, Header image: https://cdn.akamai.steamstatic.com/steam/apps/606720/header.jpg?t=1558493716, Genres: Action,Casual,Indie,Sports, Combined Score: 6.451183656726153e-08
Input Game: Dr. Tacocat, Recommendation 3: Name: Master Pyrox Wizard Smackdown, Header image: https://cdn.akamai.steamstatic.com/steam/apps/861960/header.jpg?t=1553625831, Genres: Action,Casual,Indie,Sports, Combined Score: 6.451183656726153e-08
Input Game: Galactic Bowling, Recommendation 4: Name: clutchball, Header image: https://cdn.akamai.steamstatic.com/steam/apps/1167710/header.jpg?t=1639495265, Genres: Indie,Sports, Combined Score: 6.451183656726153e-08
Input Game: Circle of Sumo, Recommendation 5: Name

This code is designed to recommend similar video games based on their genres and combined scores. It uses a TF-IDF vectorization and nearest neighbors approach to find games with similar features and scores. Here are the important steps in the code:

1. **Data Preprocessing**: The code preprocesses the game data, specifically the 'Genres' and 'Combined Score' columns. It fills missing values in the 'Genres' column with empty strings and missing 'Combined Score' values with zeros. It also creates a new 'Features' column by combining 'Genres' and 'Combined Score' as strings.

2. **TF-IDF Vectorization**: It creates a TF-IDF (Term Frequency-Inverse Document Frequency) vectorizer called `tfidf_vectorizer_combined`. This vectorizer is used to transform the 'Features' column into a numerical representation suitable for similarity calculations. It converts the text data into a numerical format that the machine learning model can work with.

3. **Nearest Neighbors Model**: The code builds a nearest neighbors model named `knn_combined` using the NearestNeighbors class from scikit-learn. This model uses the cosine distance metric and is fitted with the TF-IDF transformed data. It is designed to find the most similar games based on the features and scores.

4. **Recommendation Function**: The `recommend_similar_games` function takes a list of input games, and for each input game, it performs the following steps:
   - Find the index of the given input game in the DataFrame.
   - Use the nearest neighbors model to find the most similar games based on combined features.
   - Sort the recommendations by the "Combined Score" in descending order.
   - Filter out the input games themselves and any duplicates from previous recommendations.
   - Append up to 5 unique recommendations to the `recommended_games` list.

5. **Example Usage**: The code provides an example usage with three input games: "Galactic Bowling," "Circle of Sumo," and "Dr. Tacocat." It then calls the `recommend_similar_games` function with these input games. The recommendations are printed, showing the input game and its top recommendations. If no recommendations are found, it displays a message indicating that no recommendations were found for the input games.

The code ensures that there are up to 5 recommendations for each input game and that duplicates of input games or previous recommendations are filtered out, resulting in a list of unique game recommendations.